<h1>Copy raw data in your S3 bucket</h1>

In [1]:
import boto3
import sagemaker
import time


role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print(region)
print(role)
 
# Replace initials placeholder.
#initials = '[initials]'
initials = 'ad82'
bucket_name = 'endtoendml-workshop-{0}'.format(initials)
prefix = '02'

print(bucket_name)

s3 = boto3.resource('s3')

copy_source = {
    'Bucket': 'gianpo-public',
    'Key': 'windturbine_raw_data.csv'
}

file_name = 'windturbine_raw_data.csv'
file_key = '{0}/data/{1}'.format(prefix, file_name)
s3.Bucket(bucket_name).copy(copy_source, file_key)
#s3.Bucket(bucket_name).download_file(file_key, file_name)

eu-west-1
arn:aws:iam::041631420165:role/service-role/AmazonSageMaker-ExecutionRole-20180507T143636
endtoendml-workshop-ad82


<h1>Create a Glue Crawler to infer schema for your data</h1>

NOTE: create Glue role in module 01

NOTE: SageMaker role has to have permissions for Glue Full Access

In [2]:
glue_client = boto3.client('glue')
response = glue_client.create_database(DatabaseInput={'Name': 'endtoendml-db'})
response = glue_client.get_database(Name='endtoendml-db')
response
assert response['Database']['Name'] == 'endtoendml-db'


In [3]:
response = glue_client.create_crawler(
    Name='endtoendml-crawler',
    Role='AWSGlueServiceRole', ## VEDI NOTA SU!!
    DatabaseName='endtoendml-db',
    Targets={'S3Targets': [{'Path': '{0}/{1}/data/'.format(bucket_name, prefix)}]}
)

In [4]:
glue_client.start_crawler(Name='endtoendml-crawler')

{'ResponseMetadata': {'RequestId': '99e8d4cf-c8b1-11e9-9c67-a9bdf8775498',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 27 Aug 2019 10:01:24 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '99e8d4cf-c8b1-11e9-9c67-a9bdf8775498'},
  'RetryAttempts': 0}}

In [5]:
glue_client.get_crawler_metrics(CrawlerNameList=['endtoendml-crawler'])

{'CrawlerMetricsList': [{'CrawlerName': 'endtoendml-crawler',
   'TimeLeftSeconds': 0.0,
   'StillEstimating': True,
   'LastRuntimeSeconds': 0.0,
   'MedianRuntimeSeconds': 0.0,
   'TablesCreated': 0,
   'TablesUpdated': 0,
   'TablesDeleted': 0}],
 'ResponseMetadata': {'RequestId': 'a3e04082-c8b1-11e9-9559-d74bf715f71f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 27 Aug 2019 10:01:29 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '212',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a3e04082-c8b1-11e9-9559-d74bf715f71f'},
  'RetryAttempts': 0}}

In [6]:
while glue_client.get_crawler_metrics(CrawlerNameList=['endtoendml-crawler'])['CrawlerMetricsList'][0]['TablesCreated'] == 0:
    time.sleep(5)
    
assert glue_client.get_crawler_metrics(CrawlerNameList=['endtoendml-crawler'])['CrawlerMetricsList'][0]['TablesCreated'] == 1


In [7]:
!pip install pyathena

You are using pip version 10.0.1, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
import pyathena
from pyathena import connect
import pandas as pd


conn = connect(s3_staging_dir='s3://{0}/{1}/staging/'.format(bucket_name, prefix), 
               region_name='eu-west-1')

df = pd.read_sql('SELECT * FROM "endtoendml-db".data limit 8;', conn)
df


,col0,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11
0,TID003,HAWT,80,61,NaN,34,33,26,1,77,E,no
1,TID010,HAWT,85,78,36.0,28,35,43,15,62,NE,yes
2,TID007,HAWT,47,31,31.0,23,46,62,15,32,N,no
3,TID008,VAWT,73,70,38.0,8,17,66,6,80,SW,yes
4,TID003,HAWT,16,23,46.0,9,76,53,14,29,W,no
5,TID001,HAWT,78,71,30.0,11,66,79,1,81,SW,no
6,TID009,HAWT,80,25,37.0,31,40,75,4,56,NW,no
7,TID002,VAWT,59,29,37.0,10,25,83,13,55,SE,no


As you can see column names are not defined, so let's modify the AWS Glue table

In [9]:
table = glue_client.get_table(DatabaseName='endtoendml-db', Name='data')
table

{'Table': {'Name': 'data',
  'DatabaseName': 'endtoendml-db',
  'Owner': 'owner',
  'CreateTime': datetime.datetime(2019, 8, 27, 10, 2, 26, tzinfo=tzlocal()),
  'UpdateTime': datetime.datetime(2019, 8, 27, 10, 2, 26, tzinfo=tzlocal()),
  'LastAccessTime': datetime.datetime(2019, 8, 27, 10, 2, 26, tzinfo=tzlocal()),
  'Retention': 0,
  'StorageDescriptor': {'Columns': [{'Name': 'col0', 'Type': 'string'},
    {'Name': 'col1', 'Type': 'string'},
    {'Name': 'col2', 'Type': 'bigint'},
    {'Name': 'col3', 'Type': 'bigint'},
    {'Name': 'col4', 'Type': 'double'},
    {'Name': 'col5', 'Type': 'bigint'},
    {'Name': 'col6', 'Type': 'bigint'},
    {'Name': 'col7', 'Type': 'bigint'},
    {'Name': 'col8', 'Type': 'bigint'},
    {'Name': 'col9', 'Type': 'bigint'},
    {'Name': 'col10', 'Type': 'string'},
    {'Name': 'col11', 'Type': 'string'}],
   'Location': 's3://endtoendml-workshop-ad82/02/data/',
   'InputFormat': 'org.apache.hadoop.mapred.TextInputFormat',
   'OutputFormat': 'org.apache.

In [10]:
table['Table']['StorageDescriptor']['Columns'] = [{'Name': 'turbine_id', 'Type': 'string'},
                                                  {'Name': 'turbine_type', 'Type': 'string'},
                                                  {'Name': 'wind_speed', 'Type': 'double'},
                                                  {'Name': 'RPM_blade', 'Type': 'double'},
                                                  {'Name': 'col4float', 'Type': 'double'},
                                                  {'Name': 'oil_level', 'Type': 'double'},
                                                  {'Name': 'temperature', 'Type': 'double'},
                                                  {'Name': 'humidity', 'Type': 'double'},
                                                  {'Name': 'vibrations_frequency', 'Type': 'double'},
                                                  {'Name': 'pressure', 'Type': 'double'},
                                                  {'Name': 'wind_direction', 'Type': 'string'},
                                                  {'Name': 'breakdown', 'Type': 'string'}]


In [11]:
updated_table = table['Table']
updated_table.pop('DatabaseName', None)
updated_table.pop('CreateTime', None)
updated_table.pop('UpdateTime', None)
updated_table.pop('CreatedBy', None)
updated_table.pop('IsRegisteredWithLakeFormation', None)

glue_client.update_table(
    DatabaseName='endtoendml-db',
    TableInput=updated_table
)

{'ResponseMetadata': {'RequestId': 'eb78a6f8-c8b1-11e9-b768-e58882e869c5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 27 Aug 2019 10:03:29 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'eb78a6f8-c8b1-11e9-b768-e58882e869c5'},
  'RetryAttempts': 0}}

Let's try to run a first SQL query to see how are data look like

In [ ]:
df = pd.read_sql('SELECT * FROM "endtoendml-db".data limit 8;', conn)
df

Another SQL query to count how many records we have

In [ ]:
pd.read_sql('SELECT COUNT(*) FROM "endtoendml-db".data;', conn)

Let's try to see what are possible values for the field "alarm" and how frequently they occur over the entire dataset

In [ ]:
pd.read_sql('SELECT breakdown, (COUNT(breakdown) * 100.0 / (SELECT COUNT(*) FROM "endtoendml-db".data)) \
            AS percent FROM "endtoendml-db".data GROUP BY breakdown;', conn)


Now we want to see if there is a correlation between temperature and humidity. To do so we run a SQL query to select only these two columns and populate a Pandas dataframe that we will use for our analysis

In [ ]:
temp_hum_df = pd.read_sql('SELECT temperature, humidity FROM "endtoendml-db".data', conn)
temp_hum_df.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.scatter(temp_hum_df.temperature, temp_hum_df.humidity)

In [ ]:
plt.hist(temp_hum_df.humidity, bins=10)

In [ ]:
wind_rpm_df = pd.read_sql('SELECT wind_speed, rpm_blade FROM "endtoendml-db".data', conn)
plt.scatter(wind_rpm_df.wind_speed, wind_rpm_df.rpm_blade)

Note: you can go to Amazon Athena console and check for query duration under History tab: usually queries are executed in a few seconds, then it takes a while for pandas to load results into a dataframe

In [ ]:
wind_rpm_df.describe()

Now we select our entire dataset and populate a dataframe.  

In [12]:
df = pd.read_sql('SELECT * FROM "endtoendml-db".data;', conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
turbine_id              1000000 non-null object
turbine_type            1000000 non-null object
wind_speed              1000000 non-null float64
rpm_blade               1000000 non-null float64
col4float               961703 non-null float64
oil_level               1000000 non-null float64
temperature             1000000 non-null float64
humidity                1000000 non-null float64
vibrations_frequency    1000000 non-null float64
pressure                1000000 non-null float64
wind_direction          1000000 non-null object
breakdown               1000000 non-null object
dtypes: float64(8), object(4)
memory usage: 91.6+ MB


You can notice that col4float has some missing values

In [ ]:
df.describe(include=['object', 'int64', 'float64'])

In [40]:
df['turbine_type'].unique()

array(['HAWT', 'VAWT', ''], dtype=object)

In [14]:
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/python/python.zip
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/jar/mleap_spark_assembly.jar

--2019-08-27 12:11:21--  https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/python/python.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 54.231.176.168
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|54.231.176.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36872 (36K) [application/zip]
Saving to: ‘python.zip’

python.zip          100%[===================>]  36.01K  --.-KB/s    in 0.1s    

2019-08-27 12:11:22 (268 KB/s) - ‘python.zip’ saved [36872/36872]

--2019-08-27 12:11:22--  https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/jar/mleap_spark_assembly.jar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 54.231.176.168
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|54.231.176.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17319576 (17M) [application/java-archive]
Saving to: ‘mleap_spark_assembly.jar’

mleap_spark_assembl 100

In [16]:
s3.Bucket(bucket_name).upload_file('python.zip', 'dependencies/python/python.zip')
s3.Bucket(bucket_name).upload_file('mleap_spark_assembly.jar', 'dependencies/jar/mleap_spark_assembly.jar')

In [57]:
s3.Bucket(bucket_name).upload_file('my_etl.py', prefix + '/scripts/my_etl.py')

ETLJob = glue_client.create_job(Name='endtoendml-job', 
                                Role='AWSGlueServiceRole',
                                Command={
                                    'Name': 'glueetl',
                                    'ScriptLocation': 's3://{0}/{1}/scripts/my_etl.py'.format(bucket_name, prefix)
                                },
                               DefaultArguments={
                                   '--job-language': 'python',
                                   '--extra-jars' : 's3://{0}/dependencies/jar/mleap_spark_assembly.jar'.format(bucket_name),
                                   '--extra-py-files': 's3://{0}/dependencies/python/python.zip'.format(bucket_name)
                               })
glue_job_name = ETLJob['Name']
print(glue_job_name)

endtoendml-job


In [58]:
JobRun = glue_client.start_job_run(JobName=glue_job_name, 
                                  Arguments = {'--S3_BUCKET': bucket_name})
print(JobRun)


{'JobRunId': 'jr_b80fa09d760197b3a3c8796bd86957d72199a682c76ef6fa4b812fab14cc7ac5', 'ResponseMetadata': {'RequestId': 'a0c51213-c8e0-11e9-bfad-472255e18ca9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 27 Aug 2019 15:37:50 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '82', 'connection': 'keep-alive', 'x-amzn-requestid': 'a0c51213-c8e0-11e9-bfad-472255e18ca9'}, 'RetryAttempts': 0}}


In [59]:
status = glue_client.get_job_run(JobName=ETLJob['Name'], RunId=JobRun['JobRunId'])
while status['JobRun']['JobRunState'] not in ('FAILED', 'SUCCEEDED', 'STOPPED'):
    print('Job status: ' + status['JobRun']['JobRunState'])
    time.sleep(30)
    status = glue_client.get_job_run(JobName=ETLJob['Name'], RunId=JobRun['JobRunId'])
    

assert status['JobRun']['JobRunState'] == 'SUCCEEDED'
    
#This will take around 6 minutes

Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
Job status: RUNNING
